In [1]:
from rdkit import Chem
import torch
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric import loader
import os
import random
import numpy as np
import pandas as pd
import networkx as nx
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
import torch.nn.functional as F
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torch_geometric.nn import GATConv, GCNConv, GINConv, MFConv, SAGEConv
from torch_geometric.nn import global_add_pool
from torch.utils.data import random_split
import pickle
from tqdm import tqdm
from torch_geometric.utils import from_networkx
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_score, f1_score, recall_score, jaccard_score, balanced_accuracy_score

# 0 DataProcessing

## 0.1 Read the data

In [2]:
train_datapath = '../data/train_datsets.pkl'
test_datapath = '../data/test_datsets.pkl'
fr = open(train_datapath, 'rb')
train_data = pickle.load(fr)
fe = open(test_datapath, 'rb')
test_data = pickle.load(fe)

## 0.2 Extract features

In [3]:
train_data_list = {}
test_data_list = {}
for i in range(1, 9):
    binNumber = bin(i)[2:].zfill(3)
    traits_ = ''.join(binNumber)
    new_train_data = []
    new_test_data = []
    for index, train_ in enumerate(train_data):
        # Initialize data_ as an empty tensor
        train_data_ = None
        train_x_10 = train_.x[:, :10]
        train_x_16 = train_.x[:, 10:26]
        train_x_3 = train_.x[:, 26:]
        if traits_[0] == '1':
            train_data_ = train_x_10
        if traits_[1] == '1':
            train_data_ = torch.cat((train_data_, train_x_16), dim=1) if train_data_ is not None else train_x_16 
        if traits_[2] == '1':
            train_data_ = torch.cat((train_data_, train_x_3), dim=1) if train_data_ is not None else train_x_3
        new_data_ = Data(
            x=train_data_,
            edge_index=train_.edge_index,
            edge_attr=train_.edge_attr,
            y=train_.y,
            target=train_.target,
            name=train_.name,
            n_index=train_.n_index,
            subgraph_node1=train_.subgraph_node1,
            subgraph_node2=train_.subgraph_node2
        )
        new_train_data.append(new_data_)
    train_data_list[traits_] = new_train_data
    for test_ in test_data:
        # Initialize data_ as an empty tensor
        test_data_ = None
        test_x_10 = test_.x[:, :10]
        test_x_16 = test_.x[:, 10:26]
        test_x_3 = test_.x[:, 26:]
        if traits_[0] == '1':
            test_data_ = test_x_10
        if traits_[1] == '1':
            test_data_ = torch.cat((test_data_, test_x_16), dim=1) if test_data_ is not None else test_x_16 
        if traits_[2] == '1':
            test_data_ = torch.cat((test_data_, test_x_3), dim=1) if test_data_ is not None != 0 else test_x_3 
        new_data_ = Data(
            x=test_data_,
            edge_index=test_.edge_index,
            edge_attr=test_.edge_attr,
            y=test_.y,
            target=test_.target,
            name=test_.name,
            n_index=test_.n_index,
            subgraph_node1=test_.subgraph_node1,
            subgraph_node2=test_.subgraph_node2
        )
        new_test_data.append(new_data_)
    test_data_list[traits_] = new_test_data

## 0.3 Data normalization

In [4]:
def get_loader_data(train_data, test_data, batch_size):
    training_set, validation_set = random_split(train_data, [int(len(train_data) * 0.85), len(train_data) - int(len(train_data) * 0.85)], generator=torch.Generator().manual_seed(12345))
    train_loader = loader.DataLoader(training_set, batch_size, shuffle=True)
    val_loader = loader.DataLoader(validation_set, batch_size, shuffle=True)
    test_loader = loader.DataLoader(test_data, batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

## 0.4 Set constants

In [5]:
RANDOM_NUMBERS = 12345 # Random numbers
batchSize = 1

## 0.5 Set a random number

In [6]:
np.set_printoptions(threshold=np.inf)
def seed_torch(seed=RANDOM_NUMBERS):
    # Seeding ensures that the same random number 
    # is generated every time you run the code.
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# 1 Set up the model

In [7]:
class ALLModel(nn.Module):
    def __init__(self, args):
        super(ALLModel, self).__init__()
        num_classes = 2

        conv_hidden = args['conv_hidden']
        traits_number = args['traits_number']
        cls_hidden = args['cls_hidden']
        self.layer_number = args['layer_number']
        self.device = args['device']
        self.conv = nn.ModuleList()
        if args['model_name'] != 'GIN':
            for i in range(self.layer_number):
                if i == 0:
                    self.conv.append(args['conv'](traits_number, conv_hidden))
                else:
                    self.conv.append(args['conv'](conv_hidden, conv_hidden))
        else:
            from torch.nn import Sequential as Seq, Linear as Lin, ReLU
            for i in range(self.layer_number):
                if i == 0:
                    nn_ = Seq(Lin(traits_number, conv_hidden), ReLU(), Lin(conv_hidden, conv_hidden))
                    self.conv.append(args['conv'](nn_))
                else:
                    nn_ = Seq(Lin(conv_hidden, conv_hidden), ReLU(), Lin(conv_hidden, conv_hidden))
                    self.conv.append(args['conv'](nn_))

        self.conv = self.conv.to(self.device)
        
        self.linear1 = nn.Linear(conv_hidden, cls_hidden)
        self.linear2 = nn.Linear(cls_hidden, num_classes)
        self.relu = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.5)

    def forward(self, mol):
        x = mol.x.to(self.device)
        edge_index = mol.edge_index.to(self.device)
        res = x
        for i in range(self.layer_number):
            res = self.conv[i](res, edge_index)
        
        res = self.linear1(res)
        res = self.relu(res)
        res = self.drop1(res)
        res = self.linear2(res)
        return res

# 2 Set up evaluation metrics

In [8]:
def top2(output, label):
    sf = nn.Softmax(dim=1)
    preds = sf(output)
    preds = preds[:, 1]
    _, indices = torch.topk(preds, 2)
    pos_index = []
    for i in range(label.shape[0]):
        if label[i] == 1:
            pos_index.append(i)  
    for li in pos_index:
        if li in indices:
            return True
    return False

def MCC(output, label):
    tn,fp,fn,tp=confusion_matrix(label, output).ravel()
    up = (tp * tn) - (fp * fn)
    down = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
    if down == 0: down = 1e-10
    return up / down

def metrics(output, label):
    tn,fp,fn,tp=confusion_matrix(label, output).ravel()
    up = (tp * tn) - (fp * fn)
    down = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
    if down == 0: down = 1e-10
    mcc = up / down
    selectivity = tn / (tn + fp + 1e-10)
    recall = tp / (tp + fn + 1e-10)
    g_mean = (selectivity * recall) ** 0.5
    balancedAccuracy = (recall + selectivity) / 2
    return mcc, selectivity, recall, g_mean, balancedAccuracy

# 3 Model manipulation

## 3.1 Training

In [9]:
def train(args, model, training_set, optimizer, criterion):
    model.train()
    sf = nn.Softmax(dim=1)
    total_loss = 0
    all_pred = []
    all_pred_raw = []
    all_labels = []
    top2n = 0
    model = model.to(args['device'])
    for mol in training_set:
        mol = mol.to(args['device'])
        mol.x = mol.x.to(torch.float32)
        target = mol.y
        
        optimizer.zero_grad()
        output = model(mol)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        
        total_loss += loss.item()
        
        # tracking
        top2n += top2(output, target)
        all_pred.append(np.argmax(output.cpu().detach().numpy(), axis=1))
        all_pred_raw.append(sf(output)[:, 1].cpu().detach().numpy())
        all_labels.append(target.cpu().detach().numpy())

    all_pred = np.concatenate(all_pred).ravel()
    all_pred_raw = np.concatenate(all_pred_raw).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    mcc, selectivity, recall, g_mean, balanced_acc = metrics(all_pred, all_labels)
    train_data_frame = pd.DataFrame(
        [{
            'Layer': args['layer_number'],
            'Feature': args['traits_number_str'],
            'Round': args['current_epoch'],
            'Model': args['model_name'],
            'ACC': accuracy_score(all_labels, all_pred),
            'Ave_loss': total_loss / len(training_set),
            'Top2': top2n / len(training_set),
            'AUC': roc_auc_score(all_labels, all_pred_raw),
            'MCC': mcc,
            'Jaccard': jaccard_score(all_labels, all_pred),
            'Precision': precision_score(all_labels, all_pred, zero_division=args['zero_division']),
            'Recall': recall,
            'F1': f1_score(all_labels, all_pred),
            'Selectivity': selectivity,
            'G_mean': g_mean,
            'Balanced_acc': balanced_acc,
        }],
        columns=['Layer', 'Feature', 'Round', 'Model', 'ACC', 'Ave_loss', 'Top2', 'AUC', 'MCC', 'Jaccard', 'Precision', 'Recall', 'F1', 'Selectivity', 'G_mean', 'Balanced_acc'],
        index=[0],
    )
    # train_data_frame.to_csv(args['train_result_path'])
    return train_data_frame

## 3.2 Evaluation

In [10]:
def val(model, device, val_set, optimizer, criterion):
    model.eval()
    sf = nn.Softmax(dim=1)
    top2n = 0
    for mol in val_set:
        mol = mol.to(device)
        mol.x = mol.x.to(torch.float32)
        target = mol.y
        optimizer.zero_grad()
        output = model(mol).to(device)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        # tracking
        top2n += top2(output, target)
    return top2n / len(val_set)

def val2(
    args,
    model,
    val_set, 
    optimizer, 
    criterion,
):
    model.eval()
    sf = nn.Softmax(dim=1)
    total_loss = 0
    all_pred = []
    all_pred_raw = []
    all_labels = []
    top2n = 0
    for mol in val_set:
        mol = mol.to(args['device'])
        mol.x = mol.x.to(torch.float32)
        target = mol.y
        optimizer.zero_grad()
        output = model(mol)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # tracking
        top2n += top2(output, target)
        all_pred.append(np.argmax(output.cpu().detach().numpy(), axis=1))
        all_pred_raw.append(sf(output)[:, 1].cpu().detach().numpy())
        all_labels.append(target.cpu().detach().numpy())
    all_pred = np.concatenate(all_pred).ravel()
    all_pred_raw = np.concatenate(all_pred_raw).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    mcc, selectivity, recall, g_mean, balanced_acc = metrics(all_pred, all_labels)
    val_data_frame = pd.DataFrame(
        [{
            'Layer': args['layer_number'],
            'Feature': args['traits_number_str'],
            'Round': args['current_epoch'],
            'Model': args['model_name'],
            'ACC': accuracy_score(all_labels, all_pred),
            'Ave_loss': None,
            'Top2': top2n / len(val_set),
            'AUC': roc_auc_score(all_labels, all_pred_raw),
            'MCC': mcc,
            'Jaccard': jaccard_score(all_labels, all_pred),
            'Precision': precision_score(all_labels, all_pred, zero_division=args['zero_division']),
            'Recall': recall,
            'F1': f1_score(all_labels, all_pred),
            'Selectivity': selectivity,
            'G_mean': g_mean,
            'Balanced_acc': balanced_acc,
        }],
        columns=['Layer', 'Feature', 'Round', 'Model', 'ACC', 'Ave_loss', 'Top2', 'AUC', 'MCC', 'Jaccard', 'Precision', 'Recall', 'F1', 'Selectivity', 'G_mean', 'Balanced_acc'],
        index=[0],
    )
    val_data_frame.to_csv(args['val_result_path'])
    return val_data_frame

## 3.3 Prediction

In [11]:
def test(args, model, test_set):
    model.eval()
    sf = nn.Softmax(dim=1)
    all_pred = []
    all_pred_raw = []
    all_labels = []
    top2n = 0
    with torch.no_grad():
        for mol in test_set:
            mol = mol.to(args['device'])
            mol.x = mol.x.to(torch.float32)
            mol.edge_attr = mol.edge_attr.to(torch.float32)
            target = mol.y
            output = model(mol)
            # squeeze
            output = torch.squeeze(output)
            # tracking
            top2n += top2(output, target)
            all_pred.append(np.argmax(output.cpu().detach().numpy(), axis=1))
            all_pred_raw.append(sf(output)[:, 1].cpu().detach().numpy())
            all_labels.append(target.cpu().detach().numpy())
    all_pred = np.concatenate(all_pred).ravel()
    all_pred_raw = np.concatenate(all_pred_raw).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    mcc, selectivity, recall, g_mean, balanced_acc = metrics(all_pred, all_labels)
    test_data_frame = pd.DataFrame(
        [{
            'Layer': args['layer_number'],
            'Feature': args['traits_number_str'],
            'Round': args['current_epoch'],
            'Model': args['model_name'],
            'ACC': accuracy_score(all_labels, all_pred),
            'Ave_loss': None,
            'Top2': top2n / len(test_set),
            'AUC': roc_auc_score(all_labels, all_pred_raw),
            'MCC': mcc,
            'Jaccard': jaccard_score(all_labels, all_pred),
            'Precision': precision_score(all_labels, all_pred, zero_division=args['zero_division']),
            'Recall': recall,
            'F1': f1_score(all_labels, all_pred),
            'Selectivity': selectivity,
            'G_mean': g_mean,
            'Balanced_acc': balanced_acc,
        }],
        columns=[
            'Layer', 'Feature', 'Round', 'Model', 
            'ACC', 'Ave_loss', 'Top2', 'AUC', 'MCC', 
            'Jaccard', 'Precision', 'Recall', 'F1', 
            'Selectivity', 'G_mean', 'Balanced_acc'],
        index=[0],
    )
    print(
        f'[{test_data_frame["Layer"][0]}]Layer'
        f'[{test_data_frame["Feature"][0]}]Feature'
        f'[{test_data_frame["Round"][0]}]Round'
        f'[{test_data_frame["Model"][0]}]Model\n'
        # f'ACC: [{test_data_frame["ACC"][0]}]\n'
        # f'Ave_loss: [{test_data_frame["Ave_loss"][0]}]\n'
        f'Top2 Score[{test_data_frame["Top2"][0]}]'
        # f'AUC: [{test_data_frame["AUC"][0]}]\n'
        # f'MCC: [{test_data_frame["MCC"][0]}]\n'
        # f'Selectivity: [{test_data_frame["Selectivity"][0]}]\n'
        # f'Recall: [{test_data_frame["Recall"][0]}]\n'
        # f'G_mean: [{test_data_frame["G_mean"][0]}]\n'
        # f'Balanced_acc: [{test_data_frame["Balanced_acc"][0]}]\n'
        # f'F1: [{test_data_frame["F1"][0]}]\n'
        # f'Precision: [{test_data_frame["Precision"][0]}]\n'
        # f'Jaccard: [{test_data_frame["Jaccard"][0]}]'
    )
    test_data_frame.to_csv(args["test_result_path"])
    # print('test_data_frame = ', test_data_frame)

    return test_data_frame

# 4 Main function

In [12]:
def main(
    args, 
    train_loader, 
    val_loader, 
    test_loader,
    is_del=False,
):
    seed_torch(args['seed'])
    torch.manual_seed(args['seed'])
    new_model_dir = os.path.join(args['model_dir'], args['traits_number_str'])
    new_result_dir = os.path.join(args['result_dir'], args['traits_number_str'])
    if not os.path.exists(new_model_dir): os.mkdir(new_model_dir)
    if not os.path.exists(new_result_dir): os.mkdir(new_result_dir)
    test_result_dict = result_dict
    test_results_path = os.path.join(
        new_result_dir, 
        '_'+str(args['layer_number'])+args['traits_number_str']+'_test.csv')
    new_new_model_dir = os.path.join(new_model_dir, args['model_name'])
    new_new_result_dir = os.path.join(new_result_dir, args['model_name'])
    val_results_dir = os.path.join(new_new_result_dir, 'valResults')
    if not os.path.exists(new_new_model_dir): os.mkdir(new_new_model_dir)
    if not os.path.exists(new_new_result_dir): os.mkdir(new_new_result_dir)
    if not os.path.exists(val_results_dir): os.mkdir(val_results_dir)
    
    args['train_result_path'] = os.path.join(
        new_result_dir, 
        '_'+str(args['layer_number'])+'_'+args['traits_number_str']+'_'+
        args['model_name']+'_train.csv')
    val_results_path = os.path.join(
            new_new_result_dir, 
            '_'+str(args['layer_number'])+'_'+args['traits_number_str']+'_'+
            args['model_name']+'_val.csv')
    test_results_path_ = os.path.join(
            new_new_result_dir, 
            '_'+str(args['layer_number'])+'_'+args['traits_number_str']+'_'+
            args['model_name']+'_test.csv')
    args['model_path'] = os.path.join(new_new_model_dir, '_'+args['traits_number_str']+'_'+args['model_name']+'.pt')
    train_result_dict = result_dict
    val_result_dict = result_dict
    model = ALLModel(args).to(args['device'])
    weights = torch.tensor([1, args['pos_weight']], dtype=torch.float32).to(args['device'])
    loss_fn = torch.nn.CrossEntropyLoss(weight=weights)
    optimizer = torch.optim.SGD(model.parameters(), lr=args['lr'])
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
    max_top2 = 0
    max_number_ = 0
    for epoch in tqdm(
        range(1, args['epoch'] + 1), 
        total=args['epoch'], 
        desc=f'Total Progress[111/{args["traits_number_str"]}]The current model[{args["model_name"]}]'):
        args['current_epoch'] = str(epoch)
        a_train_dict = train(args, model, train_loader, optimizer, loss_fn)
        train_result_dict = pd.concat([train_result_dict, a_train_dict]).reset_index(drop=True)
        top2acc = val(model, args['device'], val_loader, optimizer, loss_fn)
        scheduler.step()
        max_number_ += 1
        if max_number_ > args['max_number']: break
        if top2acc > max_top2:
            max_number_ = 0
            max_top2 = top2acc
            args['val_result_path'] = os.path.join(
                val_results_dir, 
                args['model_name']+str(args['layer_number'])+args['current_epoch']+
                '_val.csv')
            a_val_dict = val2(args, model, val_loader, optimizer, loss_fn)
            val_result_dict = pd.concat([val_result_dict, a_val_dict]).reset_index(drop=True)
            torch.save(model.state_dict(), args['model_path'])
            a_test_dict = test(args, model, test_loader)
            test_result_dict = pd.concat([test_result_dict, a_test_dict]).reset_index(drop=True)
            test_result_dict.to_csv(test_results_path_)
    train_result_dict.to_csv(args['train_result_path'])
    val_result_dict.to_csv(val_results_path)
    model = ALLModel(args).to(args['device'])
    model.load_state_dict(torch.load(args['model_path']))
    a_test_dict = test(args, model, test_loader)
    test_result_dict = pd.concat([test_result_dict, a_test_dict]).reset_index(drop=True)
    test_result_dict.to_csv(test_results_path)
    return test_result_dict
        

# 5 Entry and exit parameter settings

## 5.1 Parameter settings

In [13]:
args = {
    'lr': 0.01,
    'epoch': 400,
    'max_number': 65,
    'current_epoch': 0,
    'seed': RANDOM_NUMBERS,
    'pos_weight': 3,
    'conv_hidden': 1024,
    'cls_hidden': 1024,
    'layer_number': 3,
    'max_degree': 5,
    'traits_number': 29,
    'zero_division': 0,
    'traits_number_str': '111',
    'conv': None,
    'device': None,
    'model_name': None,
    'model_dir': None,
    'model_path': None,
    'result_dir': None,
    'train_result_path': None,
    'val_result_path': None,
    'test_result_path': None,
}

In [14]:
from torch_geometric.nn import (
    TAGConv,
    ClusterGCNConv,
    FiLMConv,
    SAGEConv,
    TransformerConv,
    MFConv,
    GATConv,
    GCNConv,
    GINConv,
)
ModelList = [
    {'name': 'TAG', 'model': TAGConv},
    {'name': 'ClusterGCN', 'model': ClusterGCNConv},
    {'name': 'FiLM', 'model': FiLMConv},
    {'name': 'SAGE', 'model': SAGEConv},
    {'name': 'Transformer', 'model': TransformerConv},
    {'name': 'MF', 'model': MFConv},
    {'name': 'GAT', 'model': GATConv},
    {'name': 'GCN', 'model': GCNConv},
    {'name': 'GIN', 'model': GINConv},
]

## 5.2 Parameter settings

In [15]:
result_dict = pd.DataFrame(
        columns=[
            'Layer', 'Feature', 'Round', 'Model', 
            'ACC', 'Ave_loss', 'Top2', 'AUC', 'MCC', 'Jaccard', 'Precision', 'Recall', 
            'F1', 'Selectivity', 'G_mean', 'Balanced_acc'])

## 5.1 Realization

In [16]:
all_test_dict = result_dict
args['device'] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# args['device'] = torch.device('cpu')
saveDir = '../data/CharacteristicAblation'
if not os.path.exists(saveDir): os.mkdir(saveDir)
saveModelsDir = os.path.join(saveDir, 'Models')
gnnResultsDir = os.path.join(saveDir, 'Results')
if not os.path.exists(saveModelsDir): os.mkdir(saveModelsDir)
if not os.path.exists(gnnResultsDir): os.mkdir(gnnResultsDir)
args['model_dir'] = saveModelsDir
args['result_dir'] = gnnResultsDir
args['conv'] = MFConv
args['model_name'] = 'MF'
layer_number = 5
for layer in range(1, layer_number+1):
    args['layer_number'] = layer
    newSaveModelsDir = os.path.join(saveModelsDir, f'{layer}Layer')
    newGnnResultsDir = os.path.join(gnnResultsDir, f'{layer}Layer')
    if not os.path.exists(newSaveModelsDir): os.mkdir(newSaveModelsDir)
    if not os.path.exists(newGnnResultsDir): os.mkdir(newGnnResultsDir)
    args['model_dir'] = newSaveModelsDir
    args['result_dir'] = newGnnResultsDir
    for i in range(1, 8):
        if i == 0: continue
        if i == 1: continue
        if i == 3: continue
        if i == 4: continue
        if i == 7: continue
        bin_ = bin(i)[2:]
        binNumber = bin_.zfill(3)
        traits_ = ''.join(binNumber)
        args['traits_number_str'] = traits_
        trainData = train_data_list[traits_]    
        testData = test_data_list[traits_]
        args['traits_number'] = testData[0].x.shape[1]
        training_set, validation_set = random_split(
            trainData, 
            [int(len(trainData) * 0.85), len(trainData) - int(len(trainData) * 0.85)], 
            generator=torch.Generator().manual_seed(args['seed']))
        # trainLoader, valLoader, testLoader = get_loader_data(train_data, test_data, batchSize)
        trainLoader, valLoader, testLoader = get_loader_data(trainData, testData, batchSize)
        # Main function
        DataFrameSet = main(args, trainLoader, valLoader, testLoader, is_del=True)
        all_test_dict = pd.concat([all_test_dict, DataFrameSet]).reset_index(drop=True)
    all_test_dict.to_csv(os.path.join(newGnnResultsDir, f'ALL_results[{layer}Layer].csv'))
all_test_dict.to_csv(os.path.join(gnnResultsDir, 'ALL_results.csv'))
print('End')

Total Progress[111/010]The current model[MF]:   0%|          | 1/400 [00:01<10:16,  1.55s/it]

[1]Layer[010]Feature[1]Round[MF]Model
Top2 Score[0.7426470588235294]


Total Progress[111/010]The current model[MF]:   0%|          | 2/400 [00:02<09:52,  1.49s/it]

[1]Layer[010]Feature[2]Round[MF]Model
Top2 Score[0.75]


Total Progress[111/010]The current model[MF]:   1%|          | 4/400 [00:05<08:55,  1.35s/it]

...

[2]Layer[010]Feature[1]Round[MF]Model
Top2 Score[0.7426470588235294]


Total Progress[111/010]The current model[MF]:   1%|          | 4/400 [00:09<16:33,  2.51s/it]

[2]Layer[010]Feature[4]Round[MF]Model
Top2 Score[0.8014705882352942]


Total Progress[111/010]The current model[MF]:   1%|▏         | 5/400 [00:12<16:06,  2.45s/it]


KeyboardInterrupt: 